<a href="https://colab.research.google.com/github/PacktPublishing/Modern-Computer-Vision-with-PyTorch-2E/blob/main/Chapter02/save_and_load_pytorch_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

In [2]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x).float().to(device)
        self.y = torch.tensor(y).float().to(device)
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]
    def __len__(self): 
        return len(self.x)

In [4]:
ds = MyDataset(x, y)
dl = DataLoader(ds, batch_size=2, shuffle=True)

In [5]:
model = nn.Sequential(
    nn.Linear(2, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
).to(device)

In [6]:
%pip install torch_summary
from torchsummary import summary

Note: you may need to restart the kernel to use updated packages.


In [7]:
summary(model, torch.zeros(1,2));

Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 8]                   24
├─ReLU: 1-2                              [-1, 8]                   --
├─Linear: 1-3                            [-1, 1]                   9
Total params: 33
Trainable params: 33
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


In [8]:
loss_func = nn.MSELoss()
from torch.optim import SGD
opt = SGD(model.parameters(), lr = 0.001)
import time
loss_history = []
start = time.time()
for _ in range(50):
    for ix, iy in dl:
        opt.zero_grad()
        loss_value = loss_func(model(ix),iy)
        loss_value.backward()
        opt.step()
        loss_history.append(loss_value.item())
end = time.time()
print(end - start)

0.10201859474182129


### Saving

In [9]:
save_path = 'mymodel.pth'
torch.save(model.state_dict(), save_path) #This saves the MODEL too

print(model.state_dict()) #The weights are shown, labeled and saved (line above)
#!du -hsc {save_path} -->  size of the model on disk; Unix/Linux cmd to show disk usage

OrderedDict([('0.weight', tensor([[ 0.0880, -0.0743],
        [ 0.5793,  0.5828],
        [ 0.5656, -0.5103],
        [-0.0792, -0.3638],
        [ 0.1229, -0.3625],
        [-0.4236,  0.5300],
        [ 0.3870,  1.0344],
        [-0.0266, -0.6840]])), ('0.bias', tensor([-0.0302,  0.2247, -0.3059, -0.6070,  0.5101, -0.6231, -0.6089,  0.5192])), ('2.weight', tensor([[-0.1855,  0.6012, -0.2590,  0.2461,  0.3138,  0.0634,  0.9225,  0.0029]])), ('2.bias', tensor([0.0391]))])


### Loading

In [10]:
#load_path = 'mymodel.pth'
#model.load_state_dict(torch.load(load_path))

### Predictions

In [19]:
val = [[8,9],[10,11],[1.5,2.5]]
val = torch.tensor(val).float()

new_state_dict = torch.load('mymodel.pth')
model.load_state_dict(new_state_dict)  # This modifies `model` in-place

# Set the model to evaluation mode
model.eval()

Sequential(
  (0): Linear(in_features=2, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [20]:
model(val.to(device))

tensor([[17.0433],
        [21.0715],
        [ 3.9361]], grad_fn=<AddmmBackward0>)

In [ ]:
val.sum(-1)

tensor([17., 21.,  4.])